<a href="https://colab.research.google.com/github/Boonyaratt/Second-Hand-Price-Prediction-CNN/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Second-Hand-Price-Prediction-CNN

##Get file from source [here!](https://zenodo.org/records/8386668)

this file download on runtime, you must run the block below to get the dataset, Which from now took about 40++ mins!


In [1]:
!wget https://zenodo.org/records/8386668/files/circular_fashion_v1.tar.gz?download=1

--2025-08-05 02:45:14--  https://zenodo.org/records/8386668/files/circular_fashion_v1.tar.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2268791213 (2.1G) [application/octet-stream]
Saving to: ‘circular_fashion_v1.tar.gz?download=1’

on_v1.tar.gz?downlo   0%[                    ]   3.71M   486KB/s    eta 77m 7s ^C


In [2]:
import os
# Rename to remove '?download=1.1'
os.rename('/content/circular_fashion_v1.tar.gz?download=1', '/content/circular_fashion_v1.tar.gz')

!tar -xvzf /content/circular_fashion_v1.tar.gz -C /content/

sep2022/2022-09-21/front_2022_09_21_10_54_07.jpg
sep2022/2022-09-15/labels_2022_09_15_09_04_22.json
feb2023/2023-02-02/brand_2023_02_02_10_33_59.jpg
sep2022/2022-09-14/front_2022_09_14_11_36_07.jpg
sep2022/2022-09-22/front_2022_09_22_05_39_22.jpg
oct2022/2022-10-20/labels_2022_10_20_07_43_55.json
jan2023/2023-01-31/front_2023_01_31_11_04_58.jpg
oct2022/2022-10-11/labels_2022_10_11_12_25_47.json
sep2022/2022-09-29/labels_2022_09_29_07_19_28.json
feb2023/2023-02-06/brand_2023_02_06_08_53_41.jpg
oct2022/2022-10-25/front_2022_10_25_08_06_25.jpg
sep2022/2022-09-22/front_2022_09_22_06_14_05.jpg
nov2022/2022-11-07/front_2022_11_07_10_30_03.jpg
oct2022/2022-10-31/back_2022_10_31_08_05_18.jpg
oct2022/2022-10-06/brand_2022_10_06_09_51_48.jpg
oct2022/2022-10-10/back_2022_10_10_08_23_48.jpg
sep2022/2022-09-27/back_2022_09_27_12_01_09.jpg
jan2023/2023-01-26/brand_2023_01_26_12_41_55.jpg

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is 

### Dir Command


In [3]:
!ls /content/

circular_fashion_v1.tar.gz  feb2023  LICENSE  oct2022	 sample_data
dec2022			    jan2023  nov2022  README.md  sep2022


In [4]:
!ls /content/oct2022/

2022-10-03  2022-10-11	2022-10-17  2022-10-21	2022-10-27
2022-10-05  2022-10-12	2022-10-18  2022-10-24	2022-10-28
2022-10-06  2022-10-13	2022-10-19  2022-10-25	2022-10-31
2022-10-10  2022-10-14	2022-10-20  2022-10-26


In [5]:
!ls /content/dec2022/2022-12-01

back_2022_12_01_09_56_35.jpg   front_2022_12_01_10_15_13.jpg
back_2022_12_01_09_56_38.jpg   front_2022_12_01_10_19_35.jpg
back_2022_12_01_10_01_59.jpg   front_2022_12_01_12_49_46.jpg
back_2022_12_01_10_04_50.jpg   front_2022_12_01_12_52_40.jpg
back_2022_12_01_10_07_56.jpg   front_2022_12_01_12_54_58.jpg
back_2022_12_01_10_10_58.jpg   front_2022_12_01_12_57_20.jpg
back_2022_12_01_10_15_13.jpg   front_2022_12_01_13_01_33.jpg
back_2022_12_01_10_19_35.jpg   front_2022_12_01_13_04_18.jpg
back_2022_12_01_12_49_46.jpg   front_2022_12_01_13_07_23.jpg
back_2022_12_01_12_52_40.jpg   front_2022_12_01_13_09_52.jpg
back_2022_12_01_12_54_58.jpg   front_2022_12_01_13_11_53.jpg
back_2022_12_01_12_57_20.jpg   front_2022_12_01_13_13_28.jpg
back_2022_12_01_13_01_33.jpg   front_2022_12_01_13_16_21.jpg
back_2022_12_01_13_04_18.jpg   front_2022_12_01_13_18_24.jpg
back_2022_12_01_13_07_23.jpg   front_2022_12_01_13_20_26.jpg
back_2022_12_01_13_09_52.jpg   front_2022_12_01_13_25_14.jpg
back_2022_12_01_13_11_53

#Create Dataset


Make function to combine image and labeling data from dir

In [6]:
import os
import json
import pandas as pd

import os
import json
import pandas as pd

def build_dataset_from_month(month_dir):
    all_data = []

    for day in os.listdir(month_dir):
        day_path = os.path.join(month_dir, day)
        if not os.path.isdir(day_path):
            continue

        for filename in os.listdir(day_path):
            if filename.startswith("labels_") and filename.endswith(".json"):
                label_path = os.path.join(day_path, filename)

                try:
                    with open(label_path, 'r') as f:
                        label_data = json.load(f)
                except json.JSONDecodeError as e:
                    print(f"[ERROR] Failed to parse JSON in: {label_path}")
                    print(f"        Error: {e}")
                    continue
                except Exception as e:
                    print(f"[ERROR] Other error in: {label_path}")
                    print(f"        Error: {e}")
                    continue

                timestamp = filename.replace("labels_", "").replace(".json", "")

                front_img = os.path.join(day_path, f"front_{timestamp}.jpg")
                back_img = os.path.join(day_path, f"back_{timestamp}.jpg")
                brand_img = os.path.join(day_path, f"brand_{timestamp}.jpg")
                brand_img = brand_img if os.path.exists(brand_img) else None

                label_data['front_img'] = front_img
                label_data['back_img'] = back_img
                label_data['brand_img'] = brand_img
                label_data['timestamp'] = timestamp
                label_data['day'] = day

                all_data.append(label_data)

    return pd.DataFrame(all_data)


##remove error file


In [7]:
!cat /content/oct2022/2022-10-17/labels_2022_10_17_07_40_32.json

cat: /content/oct2022/2022-10-17/labels_2022_10_17_07_40_32.json: No such file or directory


In [8]:
os.remove("/content/oct2022/2022-10-17/labels_2022_10_17_07_40_32.json")
os.remove("/content/oct2022/2022-10-17/back_2022_10_17_07_40_32.png")
os.remove("/content/oct2022/2022-10-17/front_2022_10_17_07_40_32.png")
os.remove("/content/oct2022/2022-10-17/brand_2022_10_17_07_40_32.png")

FileNotFoundError: [Errno 2] No such file or directory: '/content/oct2022/2022-10-17/labels_2022_10_17_07_40_32.json'

## Create Dataframe

In [9]:
df_dec2022 = build_dataset_from_month('/content/dec2022')
df_feb2023 = build_dataset_from_month('/content/feb2023')
df_jan2023 = build_dataset_from_month('/content/jan2023')
df_nov2022 = build_dataset_from_month('/content/nov2022')
df_oct2022 = build_dataset_from_month('/content/oct2022')

df = pd.concat([df_dec2022, df_feb2023, df_jan2023, df_nov2022,df_oct2022])

##Data Explore

In [ ]:
df.tail()

In [ ]:
df['annotator']

In [ ]:
df.info()

In [ ]:
df.describe()

###Display Some Cloths Image


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
import json

def display_sample(row):
    fig, axs = plt.subplots(1, 3, figsize=(12, 4))

    for i, img_type in enumerate(['front_img', 'back_img', 'brand_img']):
        img_path = row[img_type]
        if img_path and os.path.exists(img_path):
            img = Image.open(img_path)
            axs[i].imshow(img)
            axs[i].set_title(img_type)
            axs[i].axis('off')
        else:
            axs[i].set_title(f"{img_type}\nNot Found")
            axs[i].axis('off')

    plt.suptitle(f"Date: {row['day']} | Timestamp: {row['timestamp']}")
    plt.show()

    print("Category:")
    print(json.dumps(row['category'], indent=2))
    print("Type:")
    print(json.dumps(row['type'], indent=2))
    print("size:")
    print(json.dumps(row['size'], indent=2))
    print("Colors:")
    print(json.dumps(row['colors'], indent=2))
    print("price:")
    print(json.dumps(row['price'], indent=2))


In [ ]:
for i in range(3):
  display_sample(df.iloc[i])

###Data value explore

In [10]:
df.isna().sum()

,0
brand,0
brandtext,1171
category,0
type,0
size,0
colors,0
season,0
pilling,0
condition,0
price,0


In [11]:
df = df.drop(['smell','holes','weight','comment','annotator'], axis=1)

In [12]:
def display_unique_values(df, columns=None):
    if columns is None:
        columns = df.columns
    for col in columns:
        unique_vals = df[col].unique()
        print(f"Column: {col}")
        print(f"Unique values ({len(unique_vals)}): {unique_vals}\n")

df['trend'] = df['trend'].replace({'No trend': 'None'})

display_unique_values(df, ['category', 'type','size','season','trend','brand'])



Column: category
Unique values (4): ['Ladies' 'Men' 'Children' 'Unisex']

Column: type
Unique values (32): ['Blouse' 'Dress' 'Cardigan' 'Sweater' 'Top' 'Shirt' 'Tights' 'T-shirt'
 'Skirt' 'Trousers' 'Blazer' 'Jeans' 'Vest' 'Jacket' 'Training top'
 'Shorts' 'Hoodie' 'Tank top ' 'Winter jacket' 'Rain jacket'
 'Winter trousers' 'Pajamas' 'Night gown' 'Rain trousers' 'Tunic'
 'Outerwear' 'sweater' 'Denim jacket' 'Robe' 'Tank top' 'Jacker'
 'Nightgown']

Column: size
Unique values (39): ['M ' 'XL' '40' '42' '46' 'L' '38' '50' '44' 'S ' 'None' '158' '122' '52'
 'XXL' '134' '48' 'onesize' '164' '34' '36' '140' '98' '104' '146' '110'
 '128' '56' '62' 'XS' '116' '92' '74' 'XXXL' '86' '80' 'M' 'S' 'Onesize']

Column: season
Unique values (6): ['All' 'Winter' 'Spring' 'Summer' 'Autumn' 'None']

Column: trend
Unique values (6): ['None' 'Sports' '90s' 'Denim' 'Leather' '80s']

Column: brand
Unique values (168): ['Not in the list' 'Masai' '365' 'Batistini (Dressman)' 'Flash'
 'Cappuchini' 'Isolde' '

#### Cleaning df['brand']



In [17]:
# 1. Clean brand names
def clean_brand_name(brand):
    if pd.isna(brand):
        return 'Unknown'
    brand = brand.strip()
    # แปลงตัวอักษรเป็นรูปแบบมาตรฐาน (lowercase + title case)
    brand_lower = brand.lower()

    # Mapping ชื่อแบรนด์ที่เขียนต่างกันให้อยู่ในชื่อเดียวกัน (ตัวอย่าง)
    brand_corrections = {
        'bershka (zara)': 'Zara',
        'bershka (zara)': 'Zara',
        'pull and bear': 'Zara',
        'pull and bear': 'Zara',
        'massimo dutti': 'Zara',
        'logg (h&m)': 'H&M',
        'weekday': 'H&M',
        'monki': 'H&M',
        'cos': 'H&M',
        'cos': 'H&M',
        'hamton republic (kappahl)': 'Kappahl',
        'newbie (kappahl)': 'Kappahl',
        'xlnt (kappahl)': 'Kappahl',
        'fix (lindex)': 'Lindex',
        'holly and whyte (lindex)': 'Lindex',
        'edc (esprit de corps)': 'Esprit',
        'fbsisters (new yorker)': 'New Yorker',
        'amisu (new yorker)': 'New Yorker',
        'mywear (ica)': 'MyWear',
        'mywear (ica)': 'MyWear',
        'soya/soyaconcept': 'Soya/Soyaconcept',
        'na-kd': 'Na-Kd',
        'na-kd ': 'Na-Kd',
        'new look': 'New Look',
        'new look ': 'New Look',
        'monki ': 'Monki',
        'stokhlm': 'Stockhlm',
        'stockhlm ': 'Stockhlm',
        'stkohlm': 'Stockhlm',
        'soc (stadium)': 'Soc (Stadium)',
        'soc (stadium)': 'Soc (Stadium)',
        'shirt factory': 'Shirt Factory',
        'shirt factory': 'Shirt Factory',
        'mango ': 'Mango',
        'micha ': 'Micha',
        'hollister (abercrombie & fitch)': 'Hollister (Abercomb & Fitch)',
        'frk': 'Frk',
        'craff': 'Craft',
        'free quent': 'Free Quent',
        # เพิ่มเติมตามที่เจอ
    }

    if brand_lower in brand_corrections:
        return brand_corrections[brand_lower]
    else:
        # คืนชื่อแบรนด์แบบ title case (เช่น "Nike", "Adidas")
        return brand.title()

# 2. Clean ทุกแถวก่อน map กลุ่ม
df['brand_clean'] = df['brand'].apply(clean_brand_name)

# 3. สร้าง dict mapping จาก brand_clean → group
brand_groups = {
    'Sportswear & Outdoor': ['Nike', 'Adidas', 'Puma', 'Craft', 'Cross Sportwear', 'Soc (Stadium)', 'Everest', 'Helly Hansen', 'Kari Traa', 'Stormberg', 'Tretorn', 'Hummel', 'McKinley (Intersport)', 'Peak Performance', 'Skill', 'On The Peak'],
    'Fast Fashion & High Street Retailers': ['H&M', 'Zara', 'Bikbok', 'Gina Tricot', 'Only', 'New Look', 'New Yorker', 'Cubus', 'Vero Moda', 'Vila', 'Na-Kd', 'Asos', 'Mango', 'River Island', 'Pieces', 'Forever 21', 'Monki', 'Gap', 'Lindex', 'Kappahl'],
    'Premium & Designer': ['Diesel', "Levi'S", 'Lee', 'Ralph Lauren', 'Gant', 'Nudie Jeans', 'Maison Scotch', 'Filippa K', 'Inwear', 'Second Female', 'Twist & Tango', 'Part Two', 'Carin Wester', 'Jean Paul Gaultier', 'Stenströms', 'Soaked In Luxury', 'Marc O Polo', 'Moschino', 'StyleIn', 'Object', 'Juicy Couture', 'Bondelid', 'Dagmar', 'Tiger Of Sweden', 'J.Lindeberg', 'Hunkydory', 'Hillevi', 'Jack And Jones', 'Cue', 'Esprit', 'Desigual'],
    'Niche Brands': ['Masai', 'Dressman', 'Polarn Och Pyret', 'Deval', 'Indiska', 'Park Lane', 'These Glory Days', 'Odd Molly', 'Carl William', 'Free Quent', 'Soya/Soyaconcept', 'Stockhlm', 'Gudrun Sjödén', 'Swedemount', 'Oscar Of Sweden', 'Me & I', 'Samsö', 'Dobber', 'Micha', 'East West', 'Jackpot', 'Marimekko', 'Kriss', 'Saint Tropez', 'Gerry Weber'],
    'Store Brands': ['365', 'Flash', 'Cappuchini', 'Isolde', 'MyWear', 'Shirt Factory', 'Soul River', 'Etirel', 'Cellbes', 'Melly Moda', 'Lager 157', 'Alice Bizous (Joy)', 'Dorothy Perkins', 'Cream', 'Ellos', 'Prêt', 'Pure Instinct', 'Esmara (Lidl)', 'Name It', 'Miss Milla', 'Mexx', 'Hollister (Abercomb & Fitch)', 'Fransa', 'Frk', 'Oui', 'Rosebud', 'Almia'],
    'Non-Brand': ['Unknown', 'Not Applicable', 'Junkyard', 'Rappson (Gubb)', 'Tutti Frutti', 'Melka (Gammalt Svenskt)?', 'Newbody', 'Havana']
}

brand_to_group_map = {}
for group, brands in brand_groups.items():
    for brand in brands:
        brand_to_group_map[brand] = group

# 4. Map กลุ่มแบรนด์จากชื่อสะอาด
df['brand_group'] = df['brand_clean'].map(brand_to_group_map).fillna('Other Brands')



In [18]:
df['brand_group'].value_counts()

,count
brand_group,
Non-Brand,556
Fast Fashion & High Street Retailers,546
Other Brands,381
Store Brands,172
Niche Brands,120
Premium & Designer,90
Sportswear & Outdoor,47


#### Cleaning df['type']



In [54]:
def simplify_type(df):
    df['type_clean'] = df['type'].str.lower().str.strip()

    corrections = {
        'jacker': 'jacket',
        'night gown': 'nightgown',
        'tank top ': 'tank top',
        'sweater': 'sweater',
    }
    df['type_clean'] = df['type_clean'].replace(corrections)

    type_mapping = {
        'topwear': [
            'blouse', 'shirt', 't-shirt', 'tank top', 'hoodie', 'vest',
            'sweater', 'cardigan', 'top', 'tunic', 'training top'
        ],
        'bottomwear': [
            'trousers', 'shorts', 'jeans', 'tights', 'rain trousers', 'winter trousers'
        ],
        'outerwear': [
            'jacket', 'rain jacket', 'denim jacket', 'winter jacket', 'blazer', 'outerwear'
        ],
        'dresswear': [
            'dress', 'nightgown', 'robe'
        ],
        'sleepwear': [
            'pajamas'
        ]
    }

    reverse_mapping = {}
    for group, types in type_mapping.items():
        for t in types:
            reverse_mapping[t] = group

    df['type_grouped'] = df['type_clean'].map(reverse_mapping).fillna('other')

    return df

df = simplify_type(df)

In [57]:
df['type_grouped'].value_counts()

,count
type_grouped,
other,1912


####Cleaning df['size']

In [41]:
df['size'].value_counts()

,count
size,
None,321
L,286
M,261
S,160
XL,133
M,107
S,90
38,88
40,72


In [44]:
def simplify_size(df):
    df['size_clean'] = df['size'].astype(str).str.strip().str.lower()

    # Mapping dictionary
    size_mapping = {
        'xs': ['34', '36', 'xs'],
        's': ['38', '40', 's'],
        'm': ['42', '44', 'm'],
        'l': ['46', '48', 'l'],
        'xl': ['50', '52', 'xl'],
        'xxl': ['54', '56', 'xxl', 'xxxl'],
        'onesize': ['onesize', 'one size', 'onesize'],
        'kids': ['74', '80', '86', '92', '98', '104', '110', '116', '122',
                 '128', '134', '140', '146', '158', '164'],
        'unknown': ['none', 'nan', '']
    }

    def map_size(value):
        for group, values in size_mapping.items():
            if value in values:
                return group
        return 'unknown'

    df['size_grouped'] = df['size_clean'].apply(map_size)
    return df

df = simplify_size(df)

In [45]:
df['size_grouped'].value_counts()

,count
size_grouped,
m,476
s,410
l,327
unknown,324
xl,140
xs,139
kids,48
xxl,44
onesize,4
